In [7]:
%%capture
!pip install transformers torch sklearn
!pip install evaluate
!pip install accelerate scikit-learn safetensors

In [16]:
import os
import numpy as np
import torch
import pandas as pd
import gdown
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from safetensors.torch import load_file
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import evaluate
from datasets import Dataset
from torch.utils.data import DataLoader
import shutil
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%capture
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

# Training of Roberta Model

In [17]:
# # Load data
# train_df = pd.read_csv("train_split.csv")

# # Convert label columns to float (important for BCEWithLogitsLoss)
# label_columns = ['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']
# train_df[label_columns] = train_df[label_columns].astype(float)

# # Split train_df into train and validation sets
# train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# # Convert train and validation sets to Hugging Face Dataset format
# train_dataset = datasets.Dataset.from_pandas(train_data)
# val_dataset = datasets.Dataset.from_pandas(val_data)

# # Load the tokenizer
# tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

# # Define a tokenization function
# def tokenize_function(example):
#     return tokenizer(example['text'], truncation=True, padding=True, max_length=128)

# # Tokenize datasets
# train_dataset = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)

# # Define labels as multi-hot encoding
# def encode_labels(example):
#     example['labels'] = [example['Joy'], example['Fear'], example['Anger'], example['Sadness'], example['Surprise']]
#     return example

# # Apply label encoding
# train_dataset = train_dataset.map(encode_labels)
# val_dataset = val_dataset.map(encode_labels)

# # Set up a data collator
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# # Load Roberta model with multi-label classification configuration
# model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=5, problem_type="multi_label_classification")

# # Load F1 metric with macro averaging
# f1_metric = evaluate.load("f1")

# # Define custom metric computation function
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = (logits > 0).astype(int)
#     predictions = predictions.reshape(-1)
#     labels = labels.astype(int).reshape(-1)
#     f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
#     return {"f1": f1["f1"]}

# # TrainingArguments with early stopping callback
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=1e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     save_total_limit=1,
#     greater_is_better=True,
#     report_to="none",
# )

# # Initialize Trainer with EarlyStoppingCallback
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
# )


In [5]:
# Train the model
#trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.474100,0.449991,0.743610
2,0.402900,0.400036,0.793844
3,0.400300,0.364258,0.812500
4,0.266100,0.367885,0.811051
5,0.224800,0.363891,0.820330
6,0.208900,0.365356,0.823419
7,0.189000,0.365047,0.818436
8,0.171700,0.360568,0.822253


TrainOutput(global_step=1280, training_loss=0.3049406227655709, metrics={'train_runtime': 303.0949, 'train_samples_per_second': 42.231, 'train_steps_per_second': 5.279, 'total_flos': 505186833530880.0, 'train_loss': 0.3049406227655709, 'epoch': 8.0})

**Upload model weights to GitHub**

In [6]:
# Save the trained model and tokenizer
#model.save_pretrained("./saved_model")
#tokenizer.save_pretrained("./saved_model")

# Zip the saved_model directory
#shutil.make_archive("saved_model", 'zip', "./saved_model")


'/content/saved_model.zip'

# Inference of Test Dataset

In [9]:
# Create a directory to save model files
model_path = './Roberta_Weights'
os.makedirs(model_path, exist_ok=True)

# List of URLs to download
model_files = {
    "config.json": "https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/config.json",
    "merges.txt": "https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/merges.txt",
    "model.safetensors": "https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/model.safetensors",
    "special_tokens_map.json": "https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/special_tokens_map.json",
    "tokenizer_config.json": "https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/tokenizer_config.json",
    "vocab.json": "https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/vocab.json"
}

# Download each file using wget
for filename, url in model_files.items():
    !wget -P {model_path} {url}


--2024-11-12 17:07:22--  https://github.com/SushovitNanda/nlp_assignment/raw/main/Roberta_Weights/config.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SushovitNanda/nlp_assignment/main/Roberta_Weights/config.json [following]
--2024-11-12 17:07:22--  https://raw.githubusercontent.com/SushovitNanda/nlp_assignment/main/Roberta_Weights/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975 [text/plain]
Saving to: ‘./Roberta_Weights/config.json’

config.json         100%[===================>]     975  --.-KB/s    in 0s      

2024-11-12 17:07:22 (45.0 MB/s) - ‘./Roberta_Weights/config

In [18]:

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_path)

# Load the model with the .safetensors file
state_dict = load_file(os.path.join(model_path, "model.safetensors"))
model = RobertaForSequenceClassification.from_pretrained(
    model_path,
    state_dict=state_dict
)

# Set the model to evaluation mode
model.eval()

# Load test data
test_df = pd.read_csv("test_split.csv")
label_columns = ['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']
test_df[label_columns] = test_df[label_columns].astype(float)

# Convert Pandas DataFrame to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)

# Preprocess function to tokenize the text
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Apply the preprocessing
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Prepare DataLoader with DataCollatorWithPadding to handle padding automatically
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)
test_dataset = test_dataset.remove_columns(['text'])
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

# Prediction function with F1 macro calculation
def predict(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']

            # Move tensors to GPU if available
            if torch.cuda.is_available():
                input_ids = input_ids.cuda()
                attention_mask = attention_mask.cuda()
                model.cuda()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.sigmoid(logits).cpu().numpy()  # Apply sigmoid for multi-label classification
            predictions.extend(preds)
    return predictions

# Run prediction on test dataset
preds = predict(model, test_loader)

# Convert predictions to DataFrame
preds_df = pd.DataFrame(preds, columns=label_columns)

# Post-process predictions to binary values (0 or 1 based on threshold 0.5)
binary_preds_df = (preds_df >= 0.5).astype(int)

# Extract true labels from the test DataFrame
true_labels_df = test_df[label_columns]


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [19]:
# Generate the classification report
report = classification_report(true_labels_df, binary_preds_df, target_names=label_columns)
print("Classification Report:\n", report)

# F1 macro score
f1_macro = f1_score(true_labels_df, binary_preds_df, average='macro')
print("F1 Macro Score:", f1_macro)

Classification Report:
               precision    recall  f1-score   support

         Joy       0.71      0.71      0.71        94
        Fear       0.84      0.85      0.84       232
       Anger       0.75      0.35      0.47        52
     Sadness       0.72      0.61      0.66       126
    Surprise       0.68      0.78      0.73       124

   micro avg       0.76      0.73      0.74       628
   macro avg       0.74      0.66      0.68       628
weighted avg       0.76      0.73      0.73       628
 samples avg       0.69      0.68      0.66       628

F1 Macro Score: 0.6836671628903819
